In [11]:
import re
import pandas as pd

def read_geant4_file(path):
    pattern = re.compile(
        r"G4WT\d+ > \[Event (\d+)\] \[Track (\d+)\] \[Step (\d+)\] \[(\w+)\] "
        r"Proc=(\w+) \| Pos=\(([-\d\.eE]+), ([-\d\.eE]+), ([-\d\.eE]+)\) mm \| "
        r"Time=([\d\.eE]+) ns \| StepLength=([\d\.eE]+) mm \| Edep=([\d\.eE]+) keV \| "
        r"Ekin=([\d\.eE]+) MeV(?: \| CapturedBy\(Z=(\d+), A=(\d+)\))?"
    )
    
    records = []
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            match = pattern.search(line)
            if match:
                records.append(match.groups())
    
    columns = [
        "Event", "Track", "Step", "Particle", "Process",
        "PosX_mm", "PosY_mm", "PosZ_mm",
        "Time_ns", "StepLength_mm", "Edep_keV", "Ekin_MeV",
        "Captured_Z", "Captured_A"
    ]
    df = pd.DataFrame(records, columns=columns)
    for col in columns:
        if col not in ["Particle", "Process"]:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

df1 = read_geant4_file("build/2_mev_lithium1_1000.txt")
df5 = read_geant4_file("build/2_mev_lithium5_1000.txt")

cap1 = set(df1[df1["Process"] == "nCapture"]["Event"].unique())
cap5 = set(df5[df5["Process"] == "nCapture"]["Event"].unique())

only_in_1 = cap1 - cap5
only_in_5 = cap5 - cap1

print("1/1000 katkılı dosyada capture olan ama 5/1000'de olmayan eventler:", only_in_1)
print("5/1000 katkılı dosyada capture olan ama 1/1000'de olmayan eventler:", only_in_5)

FileNotFoundError: [Errno 2] No such file or directory: 'build/2_mev_lithium1_1000.txt'

In [32]:
# 2️⃣ Karşılaştırmak istediğin event ID
event_id = 530

# 3️⃣ İki dosyada da bu event’in tüm adımlarını al
steps_1 = df1[df1["Event"] == event_id].sort_values(by=["Track", "Step"])
steps_5 = df5[df5["Event"] == event_id].sort_values(by=["Track", "Step"])

# 4️⃣ Capture olup olmadığını kontrol et
cap_1 = any(steps_1["Process"] == "nCapture")
cap_5 = any(steps_5["Process"] == "nCapture")

print(f"Event {event_id} — 1/1000 dosyasında capture: {cap_1}, 5/1000 dosyasında capture: {cap_5}")

# 5️⃣ Her iki tabloyu da göster
print("\n--- 1/1000 dosyasındaki adımlar ---")
display(steps_1)

print("\n--- 5/1000 dosyasındaki adımlar ---")
display(steps_5)

Event 530 — 1/1000 dosyasında capture: True, 5/1000 dosyasında capture: False

--- 1/1000 dosyasındaki adımlar ---


,Event,Track,Step,Particle,Process,PosX_mm,PosY_mm,PosZ_mm,Time_ns,StepLength_mm,Edep_keV,Ekin_MeV,Captured_Z,Captured_A
6044,530,1,1,Neutron,hadElastic,-0.660152,-10.0125,6.02870,0.599397,11.706000,0,1.793960,NaN,NaN
6046,530,1,2,Neutron,hadElastic,-19.729100,-21.5184,-8.01663,2.022700,26.330200,0,1.613980,NaN,NaN
6048,530,1,3,Neutron,hadElastic,-32.225300,-32.6873,-12.99080,3.018900,17.482600,0,1.273340,NaN,NaN
6050,530,1,4,Neutron,hadElastic,-30.310400,-26.8363,-11.26390,3.428970,6.393970,0,0.972877,NaN,NaN
6053,530,1,5,Neutron,hadElastic,-19.056200,21.8336,37.63260,8.556710,69.902000,0,0.572622,NaN,NaN
6057,530,1,6,Neutron,hadElastic,-18.462200,23.2477,37.77890,8.703980,1.540740,0,0.075572,NaN,NaN
6059,530,1,7,Neutron,hadElastic,-18.458200,23.2479,37.78310,8.705500,0.005767,0,0.015266,NaN,NaN
6062,530,1,8,Neutron,hadElastic,-23.145200,40.0274,57.86810,24.263800,26.588100,0,0.010282,NaN,NaN
6067,530,1,9,Neutron,hadElastic,-25.466500,41.7828,59.46940,26.632300,3.321690,0,0.009757,NaN,NaN
6069,530,1,10,Neutron,hadElastic,-26.513900,42.2523,59.95450,27.544300,1.246120,0,0.008118,NaN,NaN



--- 5/1000 dosyasındaki adımlar ---


,Event,Track,Step,Particle,Process,PosX_mm,PosY_mm,PosZ_mm,Time_ns,StepLength_mm,Edep_keV,Ekin_MeV,Captured_Z,Captured_A
6172,530,1,1,Neutron,hadElastic,-0.661589,-10.0343,6.04183,0.600702,11.731500,0,1.793960,NaN,NaN
6174,530,1,2,Neutron,hadElastic,-19.775800,-21.5675,-8.03689,2.027390,26.392800,0,1.613980,NaN,NaN
6175,530,1,3,Neutron,hadElastic,-32.304100,-32.7651,-13.02390,3.026140,17.527500,0,1.273340,NaN,NaN
6182,530,1,4,Neutron,hadElastic,-30.383600,-26.8971,-11.29190,3.437410,6.412560,0,0.972877,NaN,NaN
6187,530,1,5,Neutron,hadElastic,-19.093300,21.9292,37.76170,8.581620,70.126600,0,0.572622,NaN,NaN
6189,530,1,6,Neutron,hadElastic,-18.497600,23.3475,37.90840,8.729330,1.545280,0,0.075572,NaN,NaN
6194,530,1,7,Neutron,hadElastic,-18.493600,23.3477,37.91260,8.730850,0.005788,0,0.015266,NaN,NaN
6199,530,1,8,Neutron,hadElastic,-23.198400,40.1911,58.07420,24.348500,26.689400,0,0.010282,NaN,NaN
6202,530,1,9,Neutron,hadElastic,-25.528500,41.9532,59.68160,26.725900,3.334360,0,0.009757,NaN,NaN
6205,530,1,10,Neutron,hadElastic,-26.579900,42.4245,60.16850,27.641500,1.250870,0,0.008118,NaN,NaN


In [ ]:
##ortamda çok fazla li var yakalaması lazım evet ama nötronlar yeterince yavaşlamıyor 1 ev de artık ortamda çok da fazla li var enerjisini aktarıp scattering yapıyor.

In [5]:
import re
import pandas as pd

def read_geant4_verbose(path):
    """
    Geliştirilmiş SteppingAction çıktısını DataFrame'e dönüştürür.
    Target(Z,A), CapturedBy(Z,A), Secondaries, Escape vb. destekler.
    """
    # Ana step satırını yakalayan regex
    pattern_step = re.compile(
        r"G4WT\d+ > \[Event (\d+)\] \[Track (\d+)\] \[Step (\d+)\] \[(\w+)\]\s+"
        r"Proc=(\w+)"
        r"(?: \| Target\(Z=(\d+), A=(\d+)\)| \| Target\(unknown nucleus\))?"
        r" \| Pos=\(([-\d\.eE]+), ([-\d\.eE]+), ([-\d\.eE]+)\) mm"
        r" \| Time=([\d\.eE]+) ns"
        r" \| StepLength=([\d\.eE]+) mm"
        r" \| Edep=([\d\.eE]+) keV"
        r" \| Ekin=([\d\.eE]+) MeV"
        r"(?: \| CapturedBy\(Z=(\d+), A=(\d+)\))?"
        r"(?: \s*\| (Neutron track ended \(killed\)))?"
        r"(?: \s*\| (Escaped from geometry!))?"
    )

    # İkincil parçacık satırlarını yakalayan regex
    pattern_secondary = re.compile(
        r"\*\s*(\w+)\s*\|\s*Ekin=([\d\.eE]+)\s*MeV\s*\|\s*Pos=\(([-\d\.eE]+), ([-\d\.eE]+), ([-\d\.eE]+)\)\s*mm"
    )

    records = []
    secondaries = []

    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()

    current_event = None
    for i, line in enumerate(lines):
        match = pattern_step.search(line)
        if match:
            # yeni bir step kaydı bulundu
            rec = match.groups()
            current_event = rec[0]  # event numarasını hatırla
            records.append(rec)
        else:
            # bu satır bir secondary satırı olabilir
            sec_match = pattern_secondary.search(line)
            if sec_match and current_event is not None:
                sec_data = sec_match.groups()
                secondaries.append((current_event,) + sec_data)

    # DataFrame’ler
    columns = [
        "Event", "Track", "Step", "Particle", "Process",
        "Target_Z", "Target_A", "PosX_mm", "PosY_mm", "PosZ_mm",
        "Time_ns", "StepLength_mm", "Edep_keV", "Ekin_MeV",
        "Captured_Z", "Captured_A", "TrackEnded", "Escaped"
    ]
    df_steps = pd.DataFrame(records, columns=columns)

    df_secondaries = pd.DataFrame(
        secondaries,
        columns=["Event", "SecParticle", "SecEkin_MeV", "SecPosX_mm", "SecPosY_mm", "SecPosZ_mm"]
    )

    # Sayısal sütunları dönüştür
    for c in df_steps.columns:
        if c not in ["Particle", "Process", "TrackEnded", "Escaped"]:
            df_steps[c] = pd.to_numeric(df_steps[c], errors="coerce")
    for c in df_secondaries.columns:
        if c != "SecParticle":
            df_secondaries[c] = pd.to_numeric(df_secondaries[c], errors="coerce")

    return df_steps, df_secondaries

In [8]:
df_steps, df_secondaries = read_geant4_verbose("build/log.txt")

In [9]:
df_steps

,Event,Track,Step,Particle,Process,Target_Z,Target_A,PosX_mm,PosY_mm,PosZ_mm,Time_ns,StepLength_mm,Edep_keV,Ekin_MeV,Captured_Z,Captured_A,TrackEnded,Escaped
0,500,1,1,Neutron,hadElastic,1.0,1.0,-11.40970,-46.60340,27.64060,2.835280,55.372000,0,1.895370,NaN,NaN,None,None
1,300,1,1,Neutron,hadElastic,1.0,1.0,1.85948,-1.19501,-5.12297,0.285693,5.579480,0,0.123571,NaN,NaN,None,None
2,600,1,1,Neutron,hadElastic,1.0,1.0,-39.16000,33.80980,-18.55030,2.814230,54.961000,0,0.065393,NaN,NaN,None,None
3,400,1,1,Neutron,hadElastic,1.0,1.0,26.05530,-6.35647,66.32940,3.663470,71.546300,0,1.911790,NaN,NaN,None,None
4,0,1,1,Neutron,hadElastic,1.0,1.0,-9.27137,6.11831,4.55562,0.614761,12.006100,0,0.120124,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124280,9999,1,8,Neutron,hadElastic,1.0,1.0,-26.46470,-47.03890,-23.47620,17.801300,2.430420,0,0.002257,NaN,NaN,None,None
124281,9999,1,9,Neutron,hadElastic,1.0,1.0,-29.52440,-46.07790,-40.67860,44.433100,17.498800,0,0.001276,NaN,NaN,None,None
124282,9999,1,10,Neutron,hadElastic,1.0,1.0,-29.27990,-43.99330,-43.00000,50.767900,3.129580,0,0.000264,NaN,NaN,None,None
124283,9999,1,11,Neutron,hadElastic,1.0,1.0,-29.14550,-44.09330,-43.27200,52.188300,0.319423,0,0.000193,NaN,NaN,None,None


In [10]:
df_secondaries

,Event,SecParticle,SecEkin_MeV,SecPosX_mm,SecPosY_mm,SecPosZ_mm
0,500,proton,0.104685,-11.40970,-46.60340,27.64060
1,300,proton,1.877450,1.85948,-1.19501,-5.12297
2,600,proton,1.935660,-39.16000,33.80980,-18.55030
3,400,proton,0.088258,26.05530,-6.35647,66.32940
4,0,proton,1.880900,-9.27137,6.11831,4.55562
...,...,...,...,...,...,...
170668,9999,gamma,0.001329,4.91334,-54.18260,-18.16830
170669,9999,gamma,0.001417,4.91334,-54.18260,-18.16830
170670,9999,gamma,0.001010,4.91334,-54.18260,-18.16830
170671,9999,gamma,0.000510,4.91334,-54.18260,-18.16830
